# Switch between SQL and Python in notebooks

This notebook demonstrates how to mix SQL and Python cells in a Snowflake notebook using Snowpark. You'll:

- Use Python to explore data and transform it with DataFrames
- Use SQL for quick queries
- Switch back and forth between them, seamlessly

In [ ]:
# Create session
from snowflake.snowpark.context import get_active_session
session = get_active_session()

In a typical Python IDE, anything SQL-related has to be wrapped in `session.sql` to run within a Python workflow. 

As you can see in the next cell, this is not very clean from a code maintenance perspective — the stringified SQL is hard to read without code coloring, which can make my Notebook hard to follow.

In [ ]:
session.sql('''
---> set the Role
USE ROLE SNOWFLAKE_LEARNING_ROLE;
---> set the Warehouse
USE WAREHOUSE SNOWFLAKE_LEARNING_WH;
---> set the Database
USE DATABASE SNOWFLAKE_LEARNING_DB;
---> set the Schema
SET user_name = current_user();
SET schema_name = CONCAT($user_name, '_SWITCH_BETWEEN_SQL_AND_PYTHON');
USE SCHEMA IDENTIFIER($schema_name);
''')
session.sql('''
CREATE OR REPLACE TABLE BOOK_CATALOG AS
SELECT DATEADD(DAY, - UNIFORM (0, 180, RANDOM ()), CURRENT_DATE) AS TRANSACTION_DATE,
       CONCAT ('978',LPAD (ABS(UNIFORM (1000000000, 9999999999, RANDOM ())),10,'0')) AS ISBN,
       ABS(10-NORMAL(5, 2, RANDOM())) AS RATING,
       ABS(NORMAL(25, 30::FLOAT, RANDOM())) AS PRICE,
FROM TABLE(GENERATOR(ROWCOUNT => 100));
''')

In a Snowflake Notebook, you can create SQL cells by clicking on the `+ SQL` button at the bottom of each cell, or change the language of your cell to `SQL` on the top-left cell dropdown.

In [ ]:
---> set the Role
USE ROLE SNOWFLAKE_LEARNING_ROLE;
---> set the Warehouse
USE WAREHOUSE SNOWFLAKE_LEARNING_WH;
---> set the Database
USE DATABASE SNOWFLAKE_LEARNING_DB;
---> set the Schema
SET user_name = current_user();
SET schema_name = CONCAT($user_name, '_SWITCH_BETWEEN_SQL_AND_PYTHON');
USE SCHEMA IDENTIFIER($schema_name);

-- SQL query for generating synthetic dataset
CREATE OR REPLACE TABLE BOOK_CATALOG AS
SELECT DATEADD(DAY, - UNIFORM (0, 180, RANDOM ()), CURRENT_DATE) AS TRANSACTION_DATE,
       CONCAT ('978',LPAD (ABS(UNIFORM (1000000000, 9999999999, RANDOM ())),10,'0')) AS ISBN,
       ABS(10-NORMAL(5, 2, RANDOM())) AS RATING,
       ABS(NORMAL(25, 30::FLOAT, RANDOM())) AS PRICE,
FROM TABLE(GENERATOR(ROWCOUNT => 100));

Now, you can use Python to access this SQL-generated table as a dataframe and print it out.

In [ ]:
# Start working with my Snowflake table in Python
df = session.table("BOOK_CATALOG")
df